## GPT-2 Finetuning on Sentiment Classification

### Overview

- Compare performance of different text generation model on a sentiment detection task.
- For this, we will fine the text generation model GPT-2 on train data and report performance on the test data.
- Hence, we will also learn how to fine tune the TG models along wth how to apply these model to an example NLP task.

### Model

- Huggingface

### Dataset

- reviews_final.csv - disponível em https://www.dropbox.com/sh/oh6hbg5p9wyldfk/AAD2jw5EcP5oKaYbQ5FB55OGa?dl=0

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Download and import packages

In [ ]:

# uninstall
!pip uninstall -y wandb

# download
!pip install transformers

!pip install --upgrade accelerate

# import
import re
import json
import torch
import random
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 9.2 MB/s eta 0:00:00


### Dataset load and prep functions

In [ ]:
# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, txt_list, label_list, tokenizer, max_length):
        # define variables    
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        map_label = {0:'negative', 4: 'positive'}
        # iterate through the dataset
        for txt, label in zip(txt_list, label_list):
            # prepare the text
            prep_txt = f'<|startoftext|>Review: {txt}\nSentiment: {map_label[label]}<|endoftext|>'
            # tokenize
            encodings_dict = tokenizer(prep_txt, truncation=True,
                                       max_length=max_length, padding="max_length")
            # append to list
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            self.labels.append(map_label[label])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]

# Data load function
#def load_sentiment_dataset(tokenizer, random_seed = 1, file_path="/content/drive/My Drive/Colab/training.1600000.processed.noemoticon_edit.csv"):
def load_sentiment_dataset(tokenizer, random_seed = 1, file_path="/content/drive/My Drive/Colab/reviews_final.csv"):
    # load dataset and sample 10k reviews.
    #df = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)
    #df = pd.read_csv(file_path, header=True, on_bad_lines='skip')
    df = pd.read_csv(file_path, header=None, on_bad_lines='skip')
    #df = df[[0, 5]]
    df = df[[0, 1]]
    df.columns = ['label', 'text']
    #df = df.fillna(0)
    #df['label'] = df['label'].astype(int)
    df = df.sample(7400, random_state=0)
    #df = df[df['text'].notna()]

    ##DEBUG
    #pd.set_option('display.max_rows', None)
    #print(df)



    def pick_first_n_words(string, max_words=250): # tried a few max_words, kept 250 as max tokens was < 512
        #print(string)
        split_str = string.split()
        #print(split_str[:min(len(split_str), max_words)])
        return " ".join(split_str[:min(len(split_str), max_words)])

    df['text'] = df['text'].apply(lambda x: pick_first_n_words(x))

    print("############")
    
    
    # divide into test and train
    X_train, X_test, y_train, y_test = \
              train_test_split(df['text'].tolist(), df['label'].tolist(),
              shuffle=True, test_size=0.05, random_state=random_seed, stratify=df['label'])

    # get max length
    max_length_train = max([len(tokenizer.encode(text)) for text in X_train])
    max_length_test = max([len(tokenizer.encode(text)) for text in X_test])
    max_length = max([max_length_train, max_length_test]) + 10  #for special tokens (sos and eos) and fillers
    max_length = max(max_length, 300)
    print(f"Setting max length as {max_length}")

    # format into SentimentDataset class
    train_dataset = SentimentDataset(X_train, y_train, tokenizer, max_length=max_length)

    # return
    return train_dataset, (X_test, y_test)

### Load model and tokenizer; Call data Prep

In [ ]:
# import 
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

# model
model_name = "gpt2"
seed = 42

# seed
torch.manual_seed(seed)

In [ ]:
# iterate for N trials
for trial_no in range(3):
    
    print("Loading model...")
    # load tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>', pad_token='<|pad|>')
    model = GPT2LMHeadModel.from_pretrained(model_name).cuda()
    model.resize_token_embeddings(len(tokenizer))

    print("Loading dataset...")
    train_dataset, test_dataset = load_sentiment_dataset(tokenizer, trial_no)
    
    print("Start training...")
    training_args = TrainingArguments(output_dir='results', num_train_epochs=10, 
                                    logging_steps=10, load_best_model_at_end=True,
                                      save_strategy="no", evaluation_strategy="no", per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                    warmup_steps=100, weight_decay=0.01, logging_dir='logs')

    Trainer(model=model, args=training_args, train_dataset=train_dataset,
            eval_dataset=test_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                                  'attention_mask': torch.stack([f[1] for f in data]),
                                                                  'labels': torch.stack([f[0] for f in data])}).train()
    
    # test
    print("Start testing...")
    # eval mode on model
    _ = model.eval()

    # compute prediction on test data
    original, predicted, all_text, predicted_text = [], [], [], []
    map_label = {0:'negative', 4: 'positive'}
    for text, label in tqdm(zip(test_dataset[0], test_dataset[1])):
        # predict sentiment on test data
        prompt = f'<|startoftext|>Review: {text}\nSentiment:'
        generated = tokenizer(f"<|startoftext|> {prompt}", return_tensors="pt").input_ids.cuda()
        sample_outputs = model.generate(generated, do_sample=False, top_k=50, max_length=512, top_p=0.90, 
                temperature=0, num_return_sequences=0, pad_token_id=tokenizer.eos_token_id)
        pred_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
        # extract the predicted sentiment
        try:
            pred_sentiment = re.findall("\nSentiment: (.*)", pred_text)[-1]
        except:
            pred_sentiment = "None"
        original.append(map_label[label])
        predicted.append(pred_sentiment)
        all_text.append(text)
        predicted_text.append(pred_text)
    #transform into dataframe
    df = pd.DataFrame({'text': all_text, 'predicted': predicted, 'original': original, 'predicted_text': predicted_text})
    df.to_csv(f"result_run_{trial_no}.csv", index=False)
    # compute f1 score
    print(f1_score(original, predicted, average='macro'))

Loading model...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading dataset...
############
Setting max length as 355
Start training...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,16.527200
20,12.658100
30,8.036700
40,2.932300
50,1.679500
60,1.103400
70,1.200200
80,0.958300
90,1.316200
100,0.811600


...

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,18.945200
20,19.343000
30,9.119000
40,3.649200
50,1.779800
60,1.409200
70,0.930100
80,0.903300
90,0.723600
100,1.010200


Start testing...


370it [00:17, 20.90it/s]

0.8283644181988205


In [ ]:
df

,text,predicted,original,predicted_text
0,Existe a opção para pagar com o TICKET RESTAUR...,negative,negative,Review: Existe a opção para pagar com o TICKE...
1,Aplicativo inútil. Não consegue PAGAR um lanch...,positive,negative,Review: Aplicativo inútil. Não consegue PAGAR...
2,Rappi prime atualmente é a melhor assinatura d...,positive,positive,Review: Rappi prime atualmente é a melhor ass...
3,Experiência ruim principalmente no que diz pro...,negative,negative,Review: Experiência ruim principalmente no qu...
4,"A aplicação funciona muito bem,os parceiros é ...",positive,positive,"Review: A aplicação funciona muito bem,os par..."
...,...,...,...,...
365,Imprescindível para quem quer avaliar o que co...,negative,positive,Review: Imprescindível para quem quer avaliar...
366,Bom antes era uma beleza mais ai esses tempos ...,positive,negative,Review: Bom antes era uma beleza mais ai esse...
367,Nao resolvem problemas quando é solicitada aju...,positive,negative,Review: Nao resolvem problemas quando é solic...
368,Desde que foi comprado pela Magalu vai de mal ...,negative,negative,Review: Desde que foi comprado pela Magalu va...


In [ ]:
import os
os.makedirs("gpt2_10epochs", exist_ok=True)
model.save_pretrained("gpt2_10epochs")

In [ ]:
input_text = "o aplicativo é ruim "

# predict sentiment on test data
prompt = f'<|startoftext|>Review: {input_text}\nSentiment:'
#generated = tokenizer(f"<|startoftext|> {prompt}", return_tensors="pt").input_ids
generated = tokenizer(f"<|startoftext|> {prompt}", return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=False, top_k=50, max_length=512, top_p=0.90, 
                                temperature=0, num_return_sequences=0, pad_token_id=tokenizer.eos_token_id)
pred_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
print(pred_text)
# extract the predicted sentiment
try:
  pred_sentiment = re.findall("\nSentiment: (.*)", pred_text)[-1]
except:
  pred_sentiment = "None"

print(pred_sentiment)

 Review: o aplicativo é ruim 
Sentiment: negative
negative


In [ ]:
df

,text,predicted,original,predicted_text
0,Existe a opção para pagar com o TICKET RESTAUR...,negative,negative,Review: Existe a opção para pagar com o TICKE...
1,Aplicativo inútil. Não consegue PAGAR um lanch...,positive,negative,Review: Aplicativo inútil. Não consegue PAGAR...
2,Rappi prime atualmente é a melhor assinatura d...,positive,positive,Review: Rappi prime atualmente é a melhor ass...
3,Experiência ruim principalmente no que diz pro...,negative,negative,Review: Experiência ruim principalmente no qu...
4,"A aplicação funciona muito bem,os parceiros é ...",positive,positive,"Review: A aplicação funciona muito bem,os par..."
...,...,...,...,...
365,Imprescindível para quem quer avaliar o que co...,negative,positive,Review: Imprescindível para quem quer avaliar...
366,Bom antes era uma beleza mais ai esses tempos ...,positive,negative,Review: Bom antes era uma beleza mais ai esse...
367,Nao resolvem problemas quando é solicitada aju...,positive,negative,Review: Nao resolvem problemas quando é solic...
368,Desde que foi comprado pela Magalu vai de mal ...,negative,negative,Review: Desde que foi comprado pela Magalu va...
